In [6]:
import re
import json
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime


today = time.strftime("%Y%m%d", time.localtime())


def cate_crawler(cate_key):
    cate_url = "https://www.giftu.com.tw/ws_GetProdItemList.asmx/ShowClassSub"
    form_data = {'ClassSubId': int(cate_key.split("-")[0])}
    r = requests.post(cate_url, data=form_data)
    prod_DOMAIN = "http://www.giftu.com.tw/Product.aspx?"
    prod_urls = [prod_DOMAIN + x.replace("amp;", "") for x in re.findall('ProdId=\d+&amp;ProdItemId=\d+', r.text)]
    cate_dict[cate_key].extend(prod_urls)
    


def prod_crawler(prod_url, cate_value):

    resp = requests.get(prod_url)
    soup = BeautifulSoup(resp.text, 'html5lib')
    prod_dict = {}
    prod_dict['ID'] = 'gt' + soup.select('small')[1].text.replace("品號：", "").replace(".", "").zfill(6)
    prod_dict['category'] = cate_value
    prod_dict['href'] = prod_url
    prod_dict['name'] = soup.select('.pricetitle')[0].text
    prod_dict['price'] = soup.select('.s_price')[0].text.replace(",", "").split()[1]
    prod_dict['brand'] = [soup.select('small')[0].text + "。" + soup.select('div.writing > p')[1].text]
    prod_dict['desc'] = [soup.select('p')[0].text + soup.select('div.writing > p')[0].text]
    prod_dict['memo'] = soup.select('p')[1].text

    try:
        errorChk = "first error"
        url = "https://www.giftu.com.tw/ws_prod.asmx/ShowReply"
        resps = requests.get(url)
        comment_list = re.findall('"ds-text"&gt;(.+?)&', resps.text.replace("\n", ""))
        errorChk = "second error"
        button_url = 'https://www.giftu.com.tw/' + re.findall('"khs-btn" href="(.*)"&', resps.text)[0]
        response = requests.get(button_url)
        s = BeautifulSoup(response.text, 'html5lib')

        name = s.select('.ds-name')
        for i in range(len(name)):
            text_list = s.select('.ds-text')[i].text
            comment_list.append(text_list)
        prod_dict['comment'] = comment_list
    except:
        print(errorChk)


    p_list = ["https://www.giftu.com.tw" + soup.select('#Main_imgProduct')[0]['src']]
    i = 0
    while True:
        try:
            p_list.append(soup.select('.imgfancybox > img')[i]['src'])
            i += 1
        except IndexError:
            break
    prod_dict['picture'] = p_list
    content_list.append(prod_dict)
    print(prod_dict["name"])



if __name__ == "__main__":
    
    cate_dict = {
        '1-餐具廚具':[],
        '2-傢俱飾品':[],
        '11-書寫工具':[],
        '12-辦公用品':[],
        '13-個人配件':[],
        '24-首飾':[],
        '15-玩偶':[],
        '16-卡通經典':[]
    }
    for cate_key in cate_dict.keys():
        cate_crawler(cate_key)
        print(cate_key + " urls list done")
    

    numThread = 10
    threads = ThreadPoolExecutor(numThread)
    futures = []
    thStart = datetime.now()

    with ThreadPoolExecutor(max_workers=numThread) as executor:
    
        for cate_key in cate_dict.keys():
            cate_value = int(cate_key.split("-")[0])
            content_list = []
            for url in cate_dict[cate_key]:
                futures.append(threads.submit(prod_crawler, url, cate_value))
            
            with open('gt_prod_%s_%s.json' % (cate_value, today), 'w') as f:
                f.write(json.dumps(content_list, ensure_ascii=False, indent=4))
            print(cate_key + " done")

        
    wait(futures)
    thEnd = datetime.now()
    timeSpent = str(thEnd - thStart).split('.')[0]
        
    print("執行緒:" + str(numThread))
    
    prod_num = ""
    for cate_key in cate_dict.keys():
        prod_num += len(cate_dict[cate_key])
        
    print("商品數:" + str(prod_num))
    print("耗時:" + timeSpent)

1-餐具廚具 urls list done
2-傢俱飾品 urls list done
11-書寫工具 urls list done
12-辦公用品 urls list done
13-個人配件 urls list done
24-首飾 urls list done
15-玩偶 urls list done
16-卡通經典 urls list done
1-餐具廚具 done
2-傢俱飾品 done
11-書寫工具 done
12-辦公用品 done
13-個人配件 done
24-首飾 done
15-玩偶 done
16-卡通經典 done
GOAT STORY Goat Mug 羊角杯 酒紅色 (Original / 中杯12oz. / 真皮皮革)GOAT STORY Goat Mug 羊角杯 黑色 (Original / 中杯12oz. / 真皮皮革)

GOAT STORY Goat Mug 羊角杯 酒紅色 (Grande / 大杯16oz. / 人工皮革)
木入三分 汋飲保溫瓶。栓木GOAT STORY Goat Mug 羊角杯 酒紅色 (Original / 中杯12oz. / 人工皮革)

GOAT STORY Goat Mug 羊角杯 咖啡色 (Original / 中杯12oz. / 真皮皮革)GOAT STORY Goat Mug 羊角杯 咖啡色 (Grande / 大杯16oz. / 真皮皮革)GOAT STORY Goat Mug 羊角杯 黑色 (Grande / 大杯16oz. / 真皮皮革)


木入三分 汋飲保溫瓶。胡桃
GOAT STORY Goat Mug 羊角杯 酒紅色 (Grande / 大杯16oz. / 真皮皮革)
德國蔡司 SCHOTT ZWIESEL DIVA 波爾多紅酒對杯 禮盒裝
德國蔡司 SCHOTT ZWIESEL Dancing Party Tumblers 搖搖杯 兩杯裝
德國蔡司 SCHOTT ZWIESEL DIVA 白酒對杯 禮盒裝
德國蔡司 SCHOTT ZWIESEL Estelle 伯根地紅酒對杯德國蔡司 SCHOTT ZWIESEL DIVA 伯根地紅酒對杯 禮盒裝

木入三分 雞翅木筷(6入)
德國蔡司 SCHOTT ZWIESEL Estelle 波爾多紅酒對杯德國蔡司 SCHOTT ZW

Perrocaliente Dress Gift Set 銀色盒裝餐具組 五件組 方格
Perrocaliente Dress Gift Set 銀色盒裝餐具組 Pasta兩件組 棋盤格 (湯匙+叉子)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 五件組 波卡圓點
Perrocaliente Dress Gift Set 銀色盒裝餐具組 五件組 網狀菱格
Perrocaliente Dress Gift Set 銀色盒裝餐具組 五件組 條紋Perrocaliente Dress Gift Set 銀色盒裝餐具組 五件組 棋盤格

Perrocaliente Dress Gift Set 銀色盒裝餐具組 兩對組 方格 (湯匙*2+叉子*2)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 兩對組 波卡圓點 (湯匙*2+叉子*2)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 兩對組 條紋 (湯匙*2+叉子*2)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 兩對組 棋盤格 (湯匙*2+叉子*2)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 晚餐組 方格 (湯匙+叉子+餐刀)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 兩對組 網狀菱格 (湯匙*2+叉子*2)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 晚餐組 波卡圓點 (湯匙+叉子+餐刀)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 晚餐組 條紋 (湯匙+叉子+餐刀)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 晚餐組 棋盤格 (湯匙+叉子+餐刀)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 晚餐組 網狀菱格 (湯匙+叉子+餐刀)
Perrocaliente Dress Gift Set 銀色盒裝餐具組 點心匙＆叉兩件組 方格
Perrocaliente Dress Gift Set 銀色盒裝餐具組 點心匙＆叉兩件組 波卡圓點
Perrocaliente Dress Gift Set 銀色盒裝餐具組 點心匙＆叉兩件組 條紋
Per

Propaganda MR.P 馬克杯 已壞勿用
Propaganda MR.P 馬克杯 一屁到底
Propaganda MR.P 馬克杯 全力灌注
Propaganda MR.P 馬克杯 回味無窮
Propaganda MR.P 馬克杯 放人咬狗
Propaganda MR.P 馬克杯 放狗咬人
Propaganda MR.P 馬克杯 毒死你
Propaganda MR.P 馬克杯 飛來橫禍
Propaganda MR.P 馬克杯 愛在淌血Propaganda MR.P 馬克杯 嫁給我吧！

Propaganda MR.P 馬克杯 現世報
Propaganda MR.P 馬克杯 暫停使用
Propaganda MR.P 馬克杯 熱茶燙口
Propaganda MR.P 馬克杯 斷頭小憩
Propaganda MR.P 馬克杯 難以下嚥
Propaganda MR.P 緊密結合 酒瓶塞 綠
Propaganda MR.P 輕挺點水 矽膠杯蓋
Propaganda 口乾舌燥 杯墊-白
Propaganda 口乾舌燥 杯墊-綠
Propaganda 天使之翼 紅酒開瓶器 黑
Propaganda 求救濾茶器 白Propaganda 口乾舌燥 杯墊-黑

Propaganda 求救攪拌棒
QUALY Bai Bua Tray 荷葉托盤
QUALY Merry Tree 小樹層架 白色
QUALY OPEN 開瓶器 (紅)
QUALY OPEN 開瓶器 (黑)
QUALY S&P 胡椒鹽罐 (黑)
QUALY 小象兄弟 胡椒鹽罐組 (2入)
QUALY 小熊兄弟 胡椒鹽組 (2入)
QUALY 方塊置物盒 (紅)QUALY 方塊置物盒 (綠)

QUALY 小象糖罐
QUALY 仙人掌 調味罐組
QUALY 仙人掌研磨器 白
QUALY 冬夏滋味 調味罐
QUALY 仙人掌研磨器 黑
QUALY 冬熊夏鹿-調味罐
QUALY 北極冰桶
QUALY 北極冰-晴海藍
QUALY 冰原企鵝 冰棒盒
QUALY 冰原派對 杯腳扣環 (一套6入)
QUALY 多功能果叉組(冰原派對)
QUALY 多功能果叉組(草原派對)
QUALY 朵朵雲兒 開罐器 (白)
QUALY 朵朵雲兒 開罐器 (粉)
QUALY 朵朵雲兒 開罐器 (藍)
QUALY 朵朵雲兒-收納盒
QUALY 朵朵雲兒

賽先生科學工廠 懷舊鐵皮告示牌 GENIUS AT WORK
賽先生科學工廠 懷舊鐵皮告示牌 NO STUPID PEOPLE
賽先生科學工廠 魔法櫻花 10cm
Xcellent LED 可愛夜燈時光派對 小鹿燈板
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 北極熊
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 台灣黑熊
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 企鵝
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 鹿
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 無尾熊
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 熊貓
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 樹蛙
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 貓頭鷹
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 獼猴
【12/1~12/7聖誕節限量贈送多一件燈板！】Xcellent LED 可愛夜燈時光派對 藍鵲
【11/30~12/6周年慶特選品特價】Xcellent LED 時尚桌燈 喜樂 氣質米
【11/30~12/6周年慶特選品特價】Xcellent LED 時尚桌燈 喜樂 古典綠
【11/30~12/6周年慶特選品特價】Xcellent LED 時尚桌燈 喜樂 純淨白
【11/30~12/6周年慶特選品特價】Xcellent LED 時尚桌燈 喜樂 甜粉紅
Xcellent LED 情境燈飾 月影松
Xcellent LED 情境燈飾 迎客松
Xcellent LED 情境燈飾 香荷 S
Xcellent LED 情境燈飾 眺望
【11/30~12/6周年慶特選品特價】Xcellent LED 情境燈飾 喜月
Xcellent LED 情境燈飾 畫春
Xcellent LED 情境燈飾 雲開見月
【11/30~12/6周年慶特選品特價】Xcellent LED 情境燈飾自得
【11/

【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 15cm 金 Mercury Glass Finish
【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 17.5cm 紅 (360度系列)
【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 15cm 銀 Mercury Glass Finish
【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 17.5cm 象牙白 (360度系列)
【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 22.5cm 紅 (360度系列)
【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 22.5cm 象牙白 (360度系列)
【11/30~12/6新品獨家９折！】Veraflame 擬真火焰搖擺蠟燭 Mercury Glass Finish 20cm 銀
BRAUN 可斜放數位旅行鬧鐘 BNC009 黑
BRAUN 投射電子鐘 BNC015 白
BRAUN 投射電子鐘 BNC015 黑
BRAUN 旅行收音機鬧鐘 BNC010SRC 白
BRAUN 旅行收音機鬧鐘 BNC010SRC 黑
BRAUN 旅行鬧鐘 BNC007 白
BRAUN 旅行鬧鐘 BNC007 黑
BRAUN 數位旅行鬧鐘 BNC008 黑
ZONE Jewel 簡潔樹枝飾品架(桃)
LIULI-LIVING 連連好事 (此作品包含底座) (琥珀透)
LIULI-LIVING 天天都是星期天 (彩色6件碗)
LIULI-LIVING 華美天香（牡丹花方格燭台－小）(紅粉)
LIULI-LIVING 華美天香（牡丹花方格燭台－小）(黃粉)
LIULI-LIVING 華美天香（牡丹花方格燭台－小）(藍粉)
LIULI-LIVING大吉梨(小)—大吉大利(琥珀)
LIULI-LIVING大蘋果(小)—平平安安(琥珀)
LIULI-LIVING小福氣(相框）(金紅透)
LIULI-LIVING小福氣(相框）(藍透)
LIULI-LIVING叮嚀花–冬梅韻 (相框)(自由混色)
LIULI-LIVING叮嚀花–春水仙之歌(相框)(自由混色)
LIULI-LIVING如意娃娃(透明/白色)
LIULI-LIVING叮嚀花–秋

...

TreAsia SLIDE︱燻竹名片盒 (鈦)
TreAsia T SO1 留聲耳機(金)
TreAsia T SO1 留聲耳機(銀)
TreAsia T SO1 留聲耳機(黑)
TreAsia T SO3 留聲耳機(紅銅)
TreAsia T SO3 留聲耳機(黃銅色)TreAsia T SO3 留聲耳機(黑)

TreAsia T SO3 留聲耳機(銀色)
Rocket 吉他造型耳機線收納(紅)
Rocket 吉他造型耳機線收納(黃)
Rocket 防水行李箱套 (M)-音樂歌詞( Bulletproof)
Rocket 防水行李箱套 (M)-音樂歌詞(California Dreaming)
Rocket 防水行李箱套 (M)-音樂歌詞(Come Fly)
Rocket 防水行李箱套 (M)-音樂歌詞(Fly Me)
Rocket 防水行李箱套 (M)-音樂歌詞(Hey Ho)
Rocket 時尚托特包(Don’t stop me now)
Rocket 時尚托特包(Hot stuff)
Rocket 時尚托特包(I’m So Fancy)
Rocket 迷情化妝包(Friday I’m in Love)
Rocket 迷情化妝包(Love is in the air)
Rocket 迷情化妝包(I’m So Fancy)
Rocket 迷情化妝包(Sexy and I know it)Rockets 旅用收納袋  Sock Drugs ＆ Rocknroll

Rocket 閃電造型多功能收納繩(6入)
Rockets 旅用收納袋(I ain’t your mama)
折井 銅彩名片盒 斑紋孔雀色
折井 銅彩名片盒 斑紋青銅色
折井 銅彩名片盒 斑紋純銀色
折井 銅彩名片盒 斑紋荒銅色
折井 銅彩名片盒 斑紋黑染色
ADOLE  Gemini雙子手環 紅色
ADOLE  Gemini雙子手環 棕色
ADOLE  Gemini雙子手環 綠色
ADOLE  Gemini雙子手環 黑色
ADOLE  Gemini雙子手環 藍色
ADOLE  Gemini雙子雙色手環 紅+黑
ADOLE  Gemini雙子雙色手環 棕+綠
ADOLE  Gemini雙子雙色手環 黑+藍
ADOLE  月桂編織/牛皮水桶包 酒紅色
ADOLE  月桂編織/牛皮水桶包 煙燻粉紅色
AD

【11/30~12/6新品獨家９折！】FlipBelt 飛力跑運動收納腰帶 經典款 藍色 (XS)
【11/30~12/6新品獨家９折！】FlipBelt 飛力跑運動腰帶專用 防潑小包【11/30~12/6新品獨家９折！】FlipBelt 飛力跑運動收納腰帶 經典款 鐵灰 (S)

【11/30~12/6新品獨家９折！】FlipBelt 飛力跑運動腰帶專用 號碼布固定繩
【11/30~12/6新品獨家９折！】FlipBelt 飛力跑運動腰帶專用水壺 150ml
【11/30~12/6新品獨家９折！】FlipBelt 飛力跑運動腰帶專用水壺 300ml
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6 Plus/6S Plus (三隻彈烏克麗麗)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6 Plus/6S Plus (貴婦貓阿彩)C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6 Plus/6S Plus (氣勢鱷魚)

C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6 Plus/6S Plus (躲貓貓)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6/6S (三隻彈烏克麗麗)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6 Plus/6S Plus (聞香師，小豬梅蒂)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6/6S (貴婦貓阿彩)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6/6S (氣勢鱷魚)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6/6S (躲貓貓)
C’est Si Bon 手繪插畫系列立體浮雕iphone手機殼 iphone6/6S (聞香師，小豬梅蒂)
C’est Si Bon Epeius X系列 3用多功能手提/側背/後背休閒包 酒紅色
C’est Si Bon 手感軟木手拿包 筆電/平板/文件收納包-經典格紋II
C’est Si Bon Epeius X系列 3用多功能手提/側背/後背休閒包 深海藍
C’est Si Bon 手感軟木

BONE iPhone Slimcase 7 超纖公仔背殼 企鵝 墨黑
BONE iPhone Slimcase 7 Plus 超纖公仔背殼 企鵝 緋紅
BONE iPhone Slimcase 7 Plus 超纖公仔背殼 米奇米妮 緋紅
Bone iPhone X 泡泡保護套 企鵝
Bone iPhone X 泡泡保護套 貓咪
Bone iPhone X 泡泡保護套 鴨子
Bone iPhone X 泡泡保護套 麋鹿
Bone LinKey Lightning 充電傳輸鑰匙圈 三眼外星人
Bone LinKey Lightning 充電傳輸鑰匙圈 熊抱哥
Bone LinKey Lightning 充電傳輸鑰匙圈 美國隊長
Bone LinKey Lightning 充電傳輸鑰匙圈 企鵝 Maru
Bone LinKey Lightning 充電傳輸鑰匙圈 衛兵
Bone LinKey Lightning 充電傳輸鑰匙圈 鋼鐵人
Bone LinKey Micro USB 充電傳輸鑰匙圈 白色
Bone LinKey Micro USB 充電傳輸鑰匙圈 三眼外星人
Bone LinKey Micro USB 充電傳輸鑰匙圈 美國隊長
Bone LinKey Micro USB 充電傳輸鑰匙圈 黑色Bone LinKey Micro USB 充電傳輸鑰匙圈 粉紅色

Bone Maru 企鵝小丸公仔行動電源 6700mAh 粉紅
Bone LinKey Micro USB 充電傳輸鑰匙圈 熊抱哥
Bone LinKey Micro USB 充電傳輸鑰匙圈 鋼鐵人Bone Maru 企鵝小丸公仔行動電源 6700mAh 黑Bone Marvel 泡泡行動電源 6700mAh 美國隊長


Bone Marvel 泡泡行動電源 6700mAh 鋼鐵人
Bone Power 自行車行動電源6700mAh-紅色
Bone Power 自行車行動電源6700mAh-深藍
Bone Sport b.恐龍 泡泡行動電源 6700 黑
Bone Sport b.恐龍 泡泡行動電源 6700 紅
Bone Sport b.恐龍行動藍牙喇叭 黑
Bone Sport b.恐龍隨身碟組 紅
Bone Sport b.恐龍隨身碟組 黑
BONE 米奇公仔行動電源 6700mAh
Bone 

【11/27~12/3冬季贈禮９折優惠】Georg Jensen Mens HORSESHOE 標準扣皮帶(壓紋 90cm)
【11/27~12/3冬季贈禮９折優惠】Georg Jensen Mens HORSESHOE 標準扣皮帶(黑 95cm)
【11/27~12/3冬季贈禮９折優惠】Georg Jensen Mens SADDLE 兩面式皮帶(壓紋/黑 105cm)
【11/27~12/3冬季贈禮９折優惠】Georg Jensen Mens SADDLE 標準扣皮帶(黑 90cm)
【11/27~12/3冬季贈禮９折優惠】Georg Jensen Mens SADDLE 標準扣皮帶(黑 95cm)
【11/27~12/3冬季贈禮９折優惠】Georg Jensen Mens TRAPEZIUM 兩面式皮帶(壓紋/黑 105cm)
Georg Jensen Mens 卡夾零錢包 (棕/黃)
Georg Jensen Mens 卡夾零錢包 (黑/紫)
Georg Jensen Mens 拉鍊鑰匙包 (棕/黃)
Georg Jensen Mens 拉鍊鑰匙包 (黑/紫)
Georg Jensen Mens 長皮夾 (黑/紫)
Georg Jensen Mens 長皮夾 (棕/黃)
Georg Jensen Mens 長皮夾 (黑/黑)
Georg Jensen Mens 信封式名片夾 (棕/黃)
Georg Jensen Mens 旅行文件收納包 (棕/黃)Georg Jensen Mens 信封式名片夾 (黑/紫)

Georg Jensen Mens 旅行文件收納包 (黑/紫)
Georg Jensen Mens 旅行收納包 (棕/黃)
Georg Jensen Mens 旅行收納包 (黑/紫)Georg Jensen Mens 捷運卡夾 (棕/黃)

Georg Jensen Mens 捷運卡夾 (黑/紫)
Georg Jensen Mens 票卡夾 (棕/黃)
Georg Jensen Mens 票卡夾 (黑/紫)
Georg Jensen Mens 單把鑰匙包 (黑/紫)
Georg Jensen Mens 單把鑰匙包 (黑/紫)
Georg Jensen Mens 單把鑰匙包 (棕/黃)
Georg Jensen Mens 單把鑰匙包 (棕/黃)
Geor

TypeError: must be str, not int